In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, f1_score, precision_recall_fscore_support,
    hamming_loss, jaccard_score, roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve, confusion_matrix
)

from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
df = pd.read_csv("database.csv") 
mapping = {'L': 0, 'M': 1, 'H': 2}
df['Type'] = df['Type'].map(mapping)

X = df[['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', "Tool wear [min]"]] 
Y = df[["TWF", "HDF", "PWF", "OSF", "RNF"]] 
X_np=X.values
Y_np=Y.values

In [ ]:
X_train_temp, Y_train_temp, X_test, Y_test = iterative_train_test_split(X_np, Y_np, test_size=0.3) 
X_train, Y_train, X_val, Y_val = iterative_train_test_split(X_train_temp, Y_train_temp, test_size=0.3)
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
X_val_s= scaler.transform(X_val)

In [ ]:
# Modelo
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='sigmoid')
])